In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [78]:
#path csv file
path_data_ptt = 'ptt_data.csv'
#csv to dataframe
df_ptt = pd.read_csv('ptt_data.csv')

In [79]:
df_ptt.head()

,Time,Open,High,Low,Close,rsi1 RSI,Date Time
0,30-ธ.ค.-20,43.25,43.25,42.25,42.50,58.5883,30/12/2020
1,29-ธ.ค.-20,41.25,42.50,41.00,42.50,58.5883,29/12/2020
2,28-ธ.ค.-20,42.25,42.75,41.00,41.00,52.4209,28/12/2020
3,25-ธ.ค.-20,41.25,42.25,41.25,42.00,57.7446,25/12/2020
4,24-ธ.ค.-20,40.75,41.50,40.75,41.25,54.5285,24/12/2020


In [80]:
df_ptt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Time       2525 non-null   object 
 1   Open       2525 non-null   float64
 2   High       2525 non-null   float64
 3   Low        2525 non-null   float64
 4   Close      2525 non-null   float64
 5   rsi1 RSI   2510 non-null   float64
 6   Date Time  2525 non-null   object 
dtypes: float64(5), object(2)
memory usage: 138.2+ KB


In [81]:
#change datetime format 
df_ptt['Date Time'] = pd.to_datetime(df_ptt['Date Time'], infer_datetime_format=True, utc=False, errors='ignore')
#drop time columns
df_ptt = df_ptt.drop(columns='Time')
#prepare for backtest
df_ptt_reverse = df_ptt.reindex(index=df_ptt.index[::-1]).reset_index(drop=True)

In [82]:
df_ptt_reverse.head() 

,Open,High,Low,Close,rsi1 RSI,Date Time
0,27.5,27.5,26.3,26.9,NaN,2010-10-20
1,27.5,29.2,27.5,29.1,NaN,2010-10-19
2,29.8,30.6,29.7,30.2,NaN,2010-10-18
3,29.8,29.9,29.2,29.2,NaN,2010-10-17
4,28.8,29.1,28.4,28.9,NaN,2010-10-16


In [83]:
df_ptt_reverse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2525 entries, 0 to 2524
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Open       2525 non-null   float64       
 1   High       2525 non-null   float64       
 2   Low        2525 non-null   float64       
 3   Close      2525 non-null   float64       
 4   rsi1 RSI   2510 non-null   float64       
 5   Date Time  2525 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5)
memory usage: 118.5 KB


In [84]:
#กำหนด backtest data ปี 2011- 2020
df_backtest = df_ptt_reverse[(df_ptt_reverse['Date Time'] > '2011-01-01') & (df_ptt_reverse['Date Time'] <= '2021-01-01')].reset_index(drop=True)

In [85]:
df_backtest = df_backtest.set_index('Date Time')

In [86]:
df_backtest.head()

,Open,High,Low,Close,rsi1 RSI
Date Time,,,,,
2011-01-04,32.5,33.3,32.3,33.0,62.2145
2011-01-05,33.0,33.2,32.8,33.2,63.9708
2011-01-06,33.3,33.4,32.9,33.2,63.9708
2011-01-07,33.1,33.1,32.2,32.2,50.3888
2011-01-10,32.1,32.2,31.6,31.8,46.1665


In [87]:
#สร้าง function Backtest โดยมี กำหนด RSI < 30 ให้เปิด order  BUY และ RSI> 70 ให้เปิด order SELL
#จำนวนหุ้นที่ซื้อขายต้องเป็นจำนวนเต็ม
def RSI_Backtest(df,Money:int or float):
    #ตัวแปรสำหรับการ backtest
    Order_Open = df.copy()
    Order_Open['Position'] = [int(x) for x in range(len(Order_Open))]
    Last_Order = Order_Open['Position'].max()
    Status_Order = 'No Order'
    #list เพื่อเก็บรายละเอียดแต่ละ order 
    Position = []
    Order_Status = []
    Order_Price = []
    Balance = []
    Datetime = []
    Profit_sell = [0]
    #Backtest 
    for i in Order_Open['Position']:
        #หากหุ้นยัง  Buy อยู่ให้ ปิด Order sell ในวันสุดท้ายของช่วงที่ทำการ Backtest 
        if (i == Last_Order) and (Status_Order == 'Buy'):
            Position.append(i)
            Datetime.append(Order_Open.index[i])
            Price = Order_Open.loc[Order_Open['Position']==i]['Close'][0]
            Order_Price.append(Price)
            Money = Money + Quantity*Price
            Balance.append(Money)
        #หากไม่มีหุ่้นเมื่อเจอ rsi < 30 ให้เปิด order buy    
        elif (Order_Open.loc[Order_Open['Position']==i]['rsi1 RSI'][0]<30) and (Status_Order == 'No Order'):
            Position.append(i)
            Datetime.append(Order_Open.index[i])
            Order_Status.append('Buy')
            Price = Order_Open.loc[Order_Open['Position']==i]['Close'][0]
            Order_Price.append(Price)
            Quantity = Money//float(Price)
            Money = Money - Quantity*Price
            Balance.append(Money)
            #Balance_sell.append('-')
            Status_Order = 'Buy'       
        #หากมีหุ้นเมื่อเจอ rsi > 70 ให้เปิด order sell 
        elif (Order_Open.loc[Order_Open['Position']==i]['rsi1 RSI'][0]>70) and (Status_Order == 'Buy'):
            Position.append(i)
            Datetime.append(Order_Open.index[i])
            Order_Status.append('Sell')
            Price = Order_Open.loc[Order_Open['Position']==i]['Close'][0]
            Order_Price.append(Price)
            Money = Money + Quantity*Price
            Balance.append(Money)
            #Balance_sell.append(Money)
            Status_Order = 'No Order'
    #Dataframe ของของรายละเอียดของ order ที่ทำการ backtest
    Order = pd.DataFrame({
        'Date Time': Datetime,
        'Postion':Position,
        'Order' : Order_Status,
        'Price' : Order_Price,
        'Balance': Balance,
    })
    
    return Order

Order_PTT = RSI_Backtest(df=df_backtest,Money=500000)

In [88]:
#สรุป order ที่ทำการซื้อขายทั้งหมดตั้งแต่ปี 2011-2020 
Order_PTT

,Date Time,Postion,Order,Price,Balance
0,2011-06-08,102,Buy,33.00,17.00
1,2012-02-08,269,Sell,34.80,527271.80
2,2012-05-23,337,Buy,30.80,6.60
3,2012-12-06,473,Sell,32.80,561509.80
4,2013-11-21,709,Buy,30.10,24.40
5,2014-07-07,858,Sell,32.60,608144.80
6,2014-12-11,965,Buy,33.30,20.20
7,2015-11-04,1183,Sell,29.90,546054.00
8,2015-12-09,1207,Buy,23.20,18.80
9,2016-03-04,1265,Sell,27.70,651966.00


สรุป ณ วันสุดท้ายเงินวันสุดท้ายคือ 896366.30 บาท

In [89]:
#Profit Result
profit_ptt = list(Order_PTT.loc[Order_PTT['Order']=='Sell']['Balance'])
profit_ptt.insert(0,500000)
list_profit = []
for i in range(1,len(profit_ptt)):
    list_profit.append(profit_ptt[i]-profit_ptt[i-1])
    #dataframe ของ profit
Summary_Profit = Order_PTT.loc[Order_PTT['Order']=='Sell'].reset_index(drop=True)

Summary_Profit['Profit'] = list_profit

In [90]:
#สรุปกำไรขาดทุนของแต่ละ order ที่มีการขายเกิดขึ้น
Summary_Profit

,Date Time,Postion,Order,Price,Balance,Profit
0,2012-02-08,269,Sell,34.80,527271.80,27271.80
1,2012-12-06,473,Sell,32.80,561509.80,34238.00
2,2014-07-07,858,Sell,32.60,608144.80,46635.00
3,2015-11-04,1183,Sell,29.90,546054.00,-62090.80
4,2016-03-04,1265,Sell,27.70,651966.00,105912.00
5,2017-01-05,1467,Sell,38.70,811284.80,159318.80
6,2020-01-06,2201,Sell,47.00,751341.05,-59943.75
7,2020-06-08,2305,Sell,40.75,785054.80,33713.75
8,2020-11-10,2408,Sell,38.25,896366.30,111311.50


In [91]:
#ค่าสถิติหรือค่าที่ช่วยในการประกอบการตัดสินใจว่าโมเดลนี้นำมาใช้จริงได้หรือไม่
'''
คำนวณจากค่ากำไรขาดทุนแต่ละ order โดยค่าที่ใช้คือ mean,sd
'''
def stat_backtest(profit:list):
    sd = np.std(profit)
    mean = np.average(profit)
    return mean,sd
    
mean_ptt,sd_ptt = stat_backtest(list_profit)

print(f'ค่า mean ของ profit : {mean_ptt}',
      f'ค่า standard deviation ของ profit : {sd_ptt}')

ค่า mean ของ profit : 44040.69999999998 ค่า standard deviation ของ profit : 70087.07678728814


In [92]:
#MACD

'''
จากโจทย์หาค่า MACD(12,26,9) ปี 2011-2020
MACD  = EMA(12) - EMA(16)
Signal = EMA(MACD)
'''
df_macd = df_backtest.copy()

def macd(df,x,y,z):
    EMA_x = df['Close'].ewm(span=x, adjust=False,min_periods= x).mean()
    EMA_y = df['Close'].ewm(span=y, adjust=False,min_periods= y).mean()
    MACD_line = EMA_x - EMA_y
    Signal_line = MACD_line.ewm(span=z, adjust=False,min_periods=z).mean()
    df['macd'] = MACD_line
    df['signal_line'] = Signal_line
    return df

macd(df_macd,12,26,9)
    

,Open,High,Low,Close,rsi1 RSI,macd,signal_line
Date Time,,,,,,,
2011-01-04,32.50,33.30,32.30,33.00,62.2145,NaN,NaN
2011-01-05,33.00,33.20,32.80,33.20,63.9708,NaN,NaN
2011-01-06,33.30,33.40,32.90,33.20,63.9708,NaN,NaN
2011-01-07,33.10,33.10,32.20,32.20,50.3888,NaN,NaN
2011-01-10,32.10,32.20,31.60,31.80,46.1665,NaN,NaN
...,...,...,...,...,...,...,...
2020-12-24,40.75,41.50,40.75,41.25,54.5285,0.998016,1.454228
2020-12-25,41.25,42.25,41.25,42.00,57.7446,0.968603,1.357103
2020-12-28,42.25,42.75,41.00,41.00,52.4209,0.854748,1.256632
